In [0]:
#importing sparksession
 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
 
spark = SparkSession.builder.appName("Read").getOrCreate()

In [0]:
data = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("/FileStore/FileStore/data-2.csv")

data.display()

id,name,age
101,Sophia,28
102,Liam,35
103,Olivia,undefined
104,null,40
NA,null,null
105,Noah,-5
106,Emma,thirty


In [0]:
mycsv_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True),
    StructField("_corrupt_record",StringType(),True)
])

In [0]:
df_validated = data.withColumn("age", when(col("age") > 0, col("age")).otherwise(None))
df_validated.display()

id,name,age
101,Sophia,28
102,Liam,35
103,Olivia,null
104,null,40
NA,null,null
105,Noah,null
106,Emma,null


**PERMISSIVE**


In [0]:
data1 = spark.read.format("csv").schema(mycsv_schema).option("mode", "PERMISSIVE").load("/FileStore/FileStore/data-2.csv")
data1 = data1.withColumn("age", when(col("age") > 0, col("age")).otherwise(None))

data1.display()

id,name,age,_corrupt_record
null,name,null,"id,name,age"
101,Sophia,28,null
102,Liam,35,null
103,Olivia,null,"103,Olivia,undefined"
104,null,40,null
null,null,null,"NA,,"
105,Noah,null,null
106,Emma,null,"106,Emma,thirty"


**DROPMAL FORMED**


In [0]:
data2 = spark.read.format("csv").schema(mycsv_schema).option("mode", "DROPMALFORMED").load("/FileStore/FileStore/data-2.csv")
data2 = data2.withColumn("age", when(col("age") > 0, col("age")).otherwise(None))
data2.display()

id,name,age,_corrupt_record
101,Sophia,28,null
102,Liam,35,null
104,null,40,null
105,Noah,null,null


**FAILFAST**

In [0]:
data3 = spark.read.format("csv").schema(mycsv_schema).option("mode", "FAILFAST").load("/FileStore/FileStore/data-2.csv")
data3 = data3.withColumn("age", when(col("age") > 0, col("age")).otherwise(None))
data3.display()

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 26) (ip-10-172-249-211.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/FileStore/FileStore/data-2.csv.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:701)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:670)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:793)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:493)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:483)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:761)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:82)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:208)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:179)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:142)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:126)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:142)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:97)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:904)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1741)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:907)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:761)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1936)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:103)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:507)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.coll

**CORRUPT DATA STORAGE**

In [0]:
bad_data = spark.read.format("csv") \
    .schema(mycsv_schema) \
    .option("header", "TRUE") \
    .option("badRecordsPath", "/FileStore/FileStore/baddata") \
    .load("/FileStore/FileStore/data.csv")

bad_data = bad_data.withColumn("age", when(col("age") > 0, col("age")).otherwise(None))

bad_data.display()

id,name,age,_corrupt_record
101,Sophia,28,null
102,Liam,35,null
104,null,40,null
105,Noah,null,null


In [0]:
dbutils.fs.ls("/FileStore/FileStore/baddata/20241127T001931/bad_records/")

Out[53]: [FileInfo(path='dbfs:/FileStore/FileStore/baddata/20241127T001931/bad_records/part-00000-f01b128f-8ca7-462c-831c-2466a669ca44', name='part-00000-f01b128f-8ca7-462c-831c-2466a669ca44', size=431, modificationTime=1732666773000)]

In [0]:
baddata_read = spark.read.format("csv").load("/FileStore/FileStore/baddata/20241127T001931/bad_records/")


baddata_read.display()

_c0,_c1,_c2,_c3,_c4
"{""path"":""dbfs:/FileStore/FileStore/data.csv""","""record"":""103",Olivia,"undefined""","""reason"":""java.lang.NumberFormatException: For input string: \""undefined\""""}"
"{""path"":""dbfs:/FileStore/FileStore/data.csv""","""record"":""NA",null,""",""reason"":""java.lang.NumberFormatException: For input string: \""NA\""""}",null
"{""path"":""dbfs:/FileStore/FileStore/data.csv""","""record"":""106",Emma,"thirty""","""reason"":""java.lang.NumberFormatException: For input string: \""thirty\""""}"
